In [1]:
# !pip install transformers accelerate
# !pip install transformers[sentencepiece]
# !pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
# !pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
label2id = {'positive':0,'negative':1,'Adaptive':2,'Perfective':3,'Corrective':4}

In [3]:
test_II = pd.read_csv(r'dataset II/test.csv', encoding='utf_8_sig')
test_II.fillna('', inplace=True)
test_II = test_II[['label','text']].reset_index(drop=True)

In [4]:
test = test_II
test = test.replace({'label':label2id})

In [5]:
from openprompt.data_utils import InputExample

dataset = {}

dataset['test'] = []
for index, row in test.iterrows():
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)
print(dataset['test'][0])

{
  "guid": null,
  "label": 3,
  "meta": {},
  "text_a": "Updated property page to match functionality of the preferences page.\n",
  "text_b": "",
  "tgt_text": null
}



In [6]:
len(test)

1776

In [7]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "5-PKG_II2I")

In [8]:
# plm

In [9]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['test'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' Updated property page to match functionality of the preferences page.\n', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 3}]


In [10]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 6130, 26, 785, 543, 12, 1588, 6730, 13, 8, 11177, 543, 5, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁Update', 'd', '▁property', '▁page', '▁to', '▁match', '▁functionality', '▁of', '▁the', '▁pref

In [11]:
model_inputs = {}
for split in ['test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors


In [12]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader


In [13]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch
# this is the tricky part for multi-task, the Verbalizer is flexiable to change. 
# So We don't have to be fixed in tokenizer.
# for example the verbalizer contains multiple label words in each class
# myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
#                         label_words=[["Adaptive"], ["Perfective"], ["Corrective"]])
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=5).from_file("knowledgeable_verbalizer-all.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


##Num of label words for each label: [21, 20, 19, 20, 20]
Parameter containing:
tensor([[[ 2451,     0,     0,     0,     0],
         [ 3614,     0,     0,     0,     0],
         [ 1346,     0,     0,     0,     0],
         [  766,     0,     0,     0,     0],
         [ 7992,     0,     0,     0,     0],
         [ 1006,    35,     0,     0,     0],
         [    3, 11005,   524,     0,     0],
         [   16,  4334,     0,     0,     0],
         [    3, 21477,     0,     0,     0],
         [   36,  5595,     0,     0,     0],
         [   16, 12388,   687,   179,     0],
         [    3, 23094,     0,     0,     0],
         [  417,     0,     0,     0,     0],
         [ 3442,     0,     0,     0,     0],
         [  143,     0,     0,     0,     0],
         [ 2210,     0,     0,     0,     0],
         [ 1669,     0,     0,     0,     0],
         [11892,    15,     0,     0,     0],
         [   21,    17,  4921,     0,     0],
         [ 4095,     0,     0,     0,     0],


In [14]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [15]:
import ipynbname
nb_fname = ipynbname.name()
print(nb_fname)

PKG_II2I-evaluation-II


In [16]:
# Evaluate
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 1776it [00:03, 572.12it/s]


In [17]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
classification_rep = classification_report(alllabels, allpreds, digits=4)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.0000    0.0000    0.0000         0
           2     0.7500    0.0051    0.0102       585
           3     0.5895    0.0943    0.1626       594
           4     0.3534    0.0687    0.1150       597

    accuracy                         0.0563      1776
   macro avg     0.3386    0.0336    0.0575      1776
weighted avg     0.5630    0.0563    0.0964      1776



/usr/local/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
